In [1]:
import h5py
import os
import numpy as np
import pandas as pd

In [3]:
os.chdir('/Users/sunnyyou/Documents/Real_Time_HAI/HPCNN/H5/')

In [4]:
f = h5py.File('3B-HHR-E.MS.MRG.3IMERG.20230906-S060000-E062959.0360.V06D.RT-H5', 'r')
print(f.keys())

<KeysViewHDF5 ['Grid']>


In [5]:
grid = f['Grid']
print(grid.keys())

<KeysViewHDF5 ['nv', 'lonv', 'latv', 'time', 'lon', 'lat', 'time_bnds', 'lon_bnds', 'lat_bnds', 'precipitationCal', 'precipitationUncal', 'randomError', 'HQprecipitation', 'HQprecipSource', 'HQobservationTime', 'IRprecipitation', 'IRkalmanFilterWeight', 'probabilityLiquidPrecipitation', 'precipitationQualityIndex']>


In [6]:
print(grid['lon'])

<HDF5 dataset "lon": shape (3600,), type "<f4">


In [7]:
max(list(grid['lat']))

89.95

In [12]:
longitude_values = np.repeat(list(grid['lon']), 1800)
latitude_values = list(grid['lat'])*3600
precipitation_values = np.array(list(grid['precipitationCal'])).flatten()

dataset = pd.DataFrame({"lon": longitude_values, "lat": latitude_values, "precipitation": precipitation_values})
dataset.columns = [grid['lon'].attrs['standard_name'].decode() + " (" + grid['lon'].attrs['units'].decode() + ")",
                   grid['lat'].attrs['standard_name'].decode() + " (" + grid['lat'].attrs['units'].decode() + ")",
                   "Precipitation (" + grid['precipitationCal'].attrs['units'].decode() + ")",]
dataset.head()

,longitude (degrees_east),latitude (degrees_north),Precipitation (mm/hr)
0,-179.949997,-89.949997,-9999.900391
1,-179.949997,-89.849998,-9999.900391
2,-179.949997,-89.750000,-9999.900391
3,-179.949997,-89.650002,-9999.900391
4,-179.949997,-89.550003,-9999.900391


In [13]:
np.amax(dataset['latitude (degrees_north)'])

89.95

In [10]:
dataset.to_csv('IMERG_20230906_0600Z.csv', index=False)

In [14]:
dataset = dataset[dataset['longitude (degrees_east)'] <= -38.0]
dataset = dataset[dataset['longitude (degrees_east)'] >= -50.1]
dataset = dataset[dataset['latitude (degrees_north)'] <= 19.6]
dataset = dataset[dataset['latitude (degrees_north)'] >= 7.5]
print(dataset.shape)
dataset.head()

(14641, 3)


,longitude (degrees_east),latitude (degrees_north),Precipitation (mm/hr)
2339175,-50.049999,7.55,1.588772
2339176,-50.049999,7.65,1.586356
2339177,-50.049999,7.75,0.938168
2339178,-50.049999,7.85,0.000000
2339179,-50.049999,7.95,0.000000


In [15]:
image = dataset.pivot(index='latitude (degrees_north)', columns='longitude (degrees_east)', values='Precipitation (mm/hr)')
image = image.T

In [16]:
image = image.iloc[:, :]
image.head()

latitude (degrees_north),7.550000,7.650000,7.750000,7.850000,7.950000,8.050000,8.150000,8.250000,8.350000,8.450000,...,18.650000,18.750000,18.850000,18.950001,19.049999,19.150000,19.250000,19.350000,19.450001,19.549999
longitude (degrees_east),,,,,,,,,,,,,,,,,,,,,
-50.049999,1.588772,1.586356,0.938168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-49.950001,1.612569,0.943854,0.943854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-49.849998,1.168363,0.790335,0.790335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-49.750000,1.171288,0.790335,0.790335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-49.650002,1.199208,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
image.to_csv('ATL_202313_TS_2023090606.csv', index=False, header = False)